### Task 2.2: WA Counties with the Largest Populations

In [60]:
import pandas as pd
import geopandas

#read files into geopandas
waShape = geopandas.read_file("saep_bg10\saep_bg10.shp")
countyName = geopandas.read_file("WashingtonFIPS.dbf")

#join the shapefile and county name file together
wa_st = pd.merge(waShape, countyName, left_on = ['COUNTYFP10'], right_on = ['FIPSCounty'], how = 'left')

#create dataframe of countyname and 2017 population
cnty_pop = wa_st[['CountyName', 'POP2017']]

#additional functions to dataframe
cnty_pop = cnty_pop.groupby(['CountyName'])
cnty_pop = cnty_pop.sum()
cnty_pop = cnty_pop.sort_values(by = ['POP2017'], ascending=False)
cnty_pop.columns = ['Population (2017)']
cnty_pop.index.names = ['County Name']

#print out county dataframe of county name and population
cnty_pop.head(10)


,Population (2017)
County Name,
King,2153700.0
Pierce,859400.0
Snohomish,789400.0
Spokane,499800.0
Clark,471000.0
Thurston,276900.0
Kitsap,264300.0
Yakima,253000.0
Whatcom,216300.0


In [ ]:
#PART 2. CREATES SHAPEFILE FOR EACH COUNTY
import sys
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.6\\bin')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.6\\arcpy')
sys.path.append('C:\\Program Files (x86)\\ArcGIS\\Desktop10.6\\ArcToolbox\\Scripts')
import arcpy

#create workspace
arcpy.env.workspace="C:\\Users\\Ginue Han\\geog458_assignment\\lab3"

#join saep_bg10.shp and WashingtonFIPS by county FIPS. add the county name field to shape file.
arcpy.JoinField_management("saep_bg10\\saep_bg10.shp","COUNTYFP10", "WashingtonFIPS.dbf", "FIPSCounty",["CountyName"])

#create a list with the fields names of the shapefile
field_names = [str(f.name) for f in arcpy.ListFields("saep_bg10\\saep_bg10.shp")]

#list to store all county names in WA
county_names =[]
#create cursor for shapefile to loop through each row
cursor = arcpy.da.SearchCursor("saep_bg10\\saep_bg10.shp", "CountyName")
#uses cursor to loop through each row in shapefile inorder to add unique countynames into countynames list
for row in cursor:
    if str(row[0]) not in county_names:
        county_names.append(str(row[0]))
del cursor

#loops through county names and uses the Select_analysis tool to select rows from saep_bg10 shapefile that are from the currently
#selected county and creates a new shapefile for that county. 
for county in county_names:
    in_features = "saep_bg10\\saep_bg10.shp"
    out_feature_class = "CountyData\\{0}.shp".format(county)
    where_clause='"CountyName" = \'{0}\''.format(county)
    arcpy.Select_analysis(in_features, out_feature_class, where_clause)